## I310D: Assignment 10: Data Bias (Coding)
### Jihan Song (js97924)

The goal of this assignment is to explore the concept of bias through querying an existing natural language processing model — specifically, the Perspective API released by Google Jigsaw. For this assignment, I will examine a dataset of internet comments and their scores, in addition to formulating my own queries and using the Perspective API client to score the toxicity of each comment.

In [25]:
import pandas as pd 

sample = pd.read_csv("Sample_labeled_data.csv")

sample.head()

,Unnamed: 0,id,comment_text,toxic
0,5,0001ea8717f6de06,Thank you for understanding I think very highl...,no
1,7,000247e83dcc1211,Dear god this site is horrible,no
2,11,0002f87b16116a7f,Somebody will invariably try to add Religion ...,no
3,13,0003e1cccfd5a40a,It says it right there that it IS a type The...,no
4,14,00059ace3e3e9a53,Before adding a new product to the list mak...,no


In [81]:
sample_toxic = sample[sample['toxic'] == "yes"]
sample_toxic = sample_toxic.reset_index(drop=True)

In [91]:
import nltk
nltk.download('punkt')

toxic_words = []
toxic_dic = {}
for i in range(len(sample_toxic)):
    toxic_words += nltk.word_tokenize(sample_toxic["comment_text"][i])

for word in toxic_words:
    if word.lower() not in toxic_dic:
        toxic_dic[word.lower()] = 1
    else:
        toxic_dic[word.lower()] += 1

toxic_dic = sorted(toxic_dic.items(), key=lambda x:x[1], reverse = True)

[nltk_data] Downloading package punkt to /Users/jihansong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [92]:
toxic_dic

[('a', 3581),
 ('the', 3367),
 ('you', 3320),
 ('is', 3173),
 ('and', 2270),
 ('to', 2248),
 ('i', 2117),
 ('of', 1564),
 ('nigger', 1467),
 ('are', 1302),
 ('that', 1115),
 ('faggot', 1049),
 ('this', 1011),
 ('boob', 1001),
 ('poop', 987),
 ('it', 982),
 ('hate', 955),
 ('gay', 941),
 ('in', 902),
 ('your', 891),
 ('youi', 886),
 ('niggers', 884),
 ('my', 815),
 ('on', 711),
 ('not', 683),
 ('penis', 624),
 ('stupid', 618),
 ('for', 595),
 ('die', 595),
 ('balls', 555),
 ('be', 527),
 ('so', 520),
 ('all', 520),
 ('with', 513),
 ('bitch', 512),
 ('like', 507),
 ('have', 498),
 ('kill', 494),
 ('must', 486),
 ('vandal', 469),
 ('me', 459),
 ('anal', 455),
 ('traitor', 447),
 ('diefuk', 444),
 ('what', 442),
 ('shit', 432),
 ('sucks', 432),
 ('niggerjew', 424),
 ('he', 415),
 ('willy', 412),
 ('cunt', 410),
 ('as', 407),
 ('was', 403),
 ('an', 398),
 ('just', 386),
 ('if', 385),
 ('suck', 384),
 ('pleasenigga', 384),
 ('u', 382),
 ('up', 382),
 ('fag', 380),
 ('but', 378),
 ('dont', 37

Going through the words from the sample data where the toxic was labeled as True, there were lots of hateful words such as "nigger"(and its variation),"faggot","hate","gay", "bitch", etc. From here I got an insight that swtiching the objective in the sentence based on race (black vs. white) might cause a bias in the model. For exmaple, when I enter "I hate white people", I get 0.82 for the toxic score; while the score is 0.854 for "I hate black people." 

Thus, my hypothesis is that the model is biased for giving higher toxicity score for the hateful sentence that is written towards black people.

In [3]:
#pip install google-api-python-client

In [80]:
from googleapiclient import discovery
import json

API_KEY = 'my_api_key'

client = discovery.build(
  "commentanalyzer",
  "v1alpha1",
  developerKey=API_KEY,
  discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
  static_discovery=False,
)

analyze_request = {
  'comment': { 'text': '“some text”' },
  'requestedAttributes': {'TOXICITY': {}}
}

response = client.comments().analyze(body=analyze_request).execute()
print(json.dumps(response, indent=2))

{
  "attributeScores": {
    "TOXICITY": {
      "spanScores": [
        {
          "begin": 0,
          "end": 11,
          "score": {
            "value": 0.013885498,
            "type": "PROBABILITY"
          }
        }
      ],
      "summaryScore": {
        "value": 0.013885498,
        "type": "PROBABILITY"
      }
    }
  },
  "languages": [
    "en"
  ],
  "detectedLanguages": [
    "en"
  ]
}


Going through 15 freshly written hateful sentence (swiching only race between Black and White for each case), I conclude that when the sentence is writeen towards black people, it gets higher toxicity score compared to when it is written toward white people. Except the race, no single word was modified. The mean score for sentence for Black people was 0.695 while the mean score for sentence for White people was 0.643. For every sentence, the model decided that it is either same or more toxic when it was for Black instead of White. 

One possible reason that the model decides the hateful sentence written toward Black people is more toxic is because Black people have their history of hates and segregations, so that it is common to have more sensitivity towards toxic contents when it comes to them. The same logic might go along with other races or other minorities such as Men vs. Women. 